In [14]:
import pm4py
from log_filter import filter_log

In [ ]:
log = pm4py.read_xes("..\..\Dataset\BPI Challenge 2017.xes")

In [ ]:
#filtered_log = filter_log(log)
#len(filtered_log)

cd "X-Processes\IS 2023 version"; py xprocesses.py -log "..\..\Dataset\BPI Challenge 2017.xes" -isl 4 -rnd 5 -gen 500 -tme 3600 -con 25 -fit 1 -prc 1 -gnl 1 -smp 1

In [ ]:
# Inductive Miner
inductive_model = pm4py.discover_petri_net_inductive(log)

# Alpha Miner
alpha_model = pm4py.discover_petri_net_alpha(log)

# Heuristic Miner (standard)
heuristic_model1 = pm4py.discover_petri_net_heuristics(
    log,
    dependency_threshold=0.5,
    and_threshold=0.65,
    loop_two_threshold=0.5
)

# Heuristic Miner (stricter dependency filter)
heuristic_model2 = pm4py.discover_petri_net_heuristics(
    log,
    dependency_threshold=0.9,
    and_threshold=0.65,
    loop_two_threshold=0.5,
)

# Balanced dependency vs. parallelism
heuristic_model3 = pm4py.discover_petri_net_heuristics(
    log,
    dependency_threshold=0.7,
    and_threshold=0.7,
    loop_two_threshold=0.6
)

# Permissive, keeps more weak dependencies
heuristic_model4 = pm4py.discover_petri_net_heuristics(
    log,
    dependency_threshold=0.4,
    and_threshold=0.6,
    loop_two_threshold=0.4,
)

# Highly selective
heuristic_model5 = pm4py.discover_petri_net_heuristics(
    log,
    dependency_threshold=0.85,
    and_threshold=0.8,
    loop_two_threshold=0.7
)

petri_nets = [
    ("Inductive Miner", *inductive_model),
    ("Alpha Miner", *alpha_model),
    ("Heuristic Miner Standard", *heuristic_model1),
    ("Heuristic Miner Strict", *heuristic_model2),
    ("Heuristic Miner Balanced", *heuristic_model3),
    ("Heuristic Miner Noise-tolerant", *heuristic_model4),
    ("Heuristic Miner Selective", *heuristic_model5),
]

In [ ]:
"""inductive_bpmn_model = pm4py.convert_to_bpmn(*inductive_model)
alpha_bpmn_model = pm4py.convert_to_bpmn(*alpha_model)
heuristic_bpmn_model1 = pm4py.convert_to_bpmn(*heuristic_model1)
heuristic_bpmn_model2 = pm4py.convert_to_bpmn(*heuristic_model2)
heuristic_bpmn_model3 = pm4py.convert_to_bpmn(*heuristic_model3)
heuristic_bpmn_model4 = pm4py.convert_to_bpmn(*heuristic_model4)
heuristic_bpmn_model5 = pm4py.convert_to_bpmn(*heuristic_model5)


bpmn_models = [
    ("Inductive Miner", inductive_bpmn_model),
    ("Alpha Miner", alpha_bpmn_model),
    ("Heuristic Miner Standard", heuristic_bpmn_model1),
    ("Heuristic Miner Strict", heuristic_bpmn_model2),
    ("Heuristic Miner Balanced", heuristic_bpmn_model3),
    ("Heuristic Miner Noise-tolerant", heuristic_bpmn_model4),
    ("Heuristic Miner Selective", heuristic_bpmn_model5),
]"""

In [ ]:
"""for name, net, im, fm in petri_nets:
    path = name + " PetriNet.jpg"
    pm4py.save_vis_petri_net(net, im, fm, path)

for name, model in bpmn_models:
    path = name + " BPMN.jpg"
    pm4py.save_vis_bpmn(model,path)"""

In [ ]:
dfg_miner, start_activities, end_activities = pm4py.discover_dfg(log)
performance_dfg_miner,start_activities,end_activities = pm4py.discover_performance_dfg(log)
pm4py.save_vis_dfg(dfg_miner, start_activities, end_activities, "DFG.jpg", format="png")
pm4py.save_vis_performance_dfg(performance_dfg_miner,start_activities,end_activities, "PERF_DFG.jpg")

In [ ]:
import pm4py
import traceback
import math
from simplicity_metrics import entropy_simplicity_petri_net, size_simplicity_metric

for name, net, im, fm in petri_nets:
    print(f"--- {name} ---")

    # ----------------------------
    # Token-based replay fitness
    # ----------------------------
    try:
        tbr_fitness = pm4py.fitness_token_based_replay(log, net, im, fm)
        print("Token-based replay fitness:")
        for k, v in tbr_fitness.items():
            print(f"  {k}: {v}")
    except Exception:
        print("Error computing token-based replay fitness:")
        traceback.print_exc()
        tbr_fitness = {}

    # ----------------------------
    # Alignment-based fitness
    # ----------------------------
    """try:
        align_fitness = pm4py.fitness_alignments(log, net, im, fm)
        print("\nAlignment-based fitness:")
        for k, v in align_fitness.items():
            print(f"  {k}: {v}")
    except Exception:
        print("Error computing alignment-based fitness:")
        traceback.print_exc()
        align_fitness = {}"""
    # ----------------------------
    # Token-based replay precision
    # ----------------------------
    try:
        tbr_precision = pm4py.precision_token_based_replay(log, net, im, fm)
        print("\nToken-based replay precision:")
        print(f"  precision: {tbr_precision}")
    except Exception:
        print("Error computing token-based replay precision:")
        traceback.print_exc()
        tbr_precision = math.nan

    # ----------------------------
    # Alignment-based precision
    # ----------------------------
    """try:
        align_precision = pm4py.precision_alignments(log, net, im, fm)
        print("\nAlignment-based precision:")
        print(f"  precision: {align_precision}")
    except Exception:
        print("Error computing alignment-based precision:")
        traceback.print_exc()
        align_precision = math.nan"""

    # ----------------------------
    # Simplicity metrics
    # ----------------------------
    try:
        simplicity_arc = pm4py.simplicity_petri_net(net, im, fm, variant="arc_degree")
    except Exception:
        print("Error computing arc-degree simplicity:")
        traceback.print_exc()
        simplicity_arc = math.nan

    """try:
        simplicity_cardoso = pm4py.simplicity_petri_net(net, im, fm, variant="extended_cardoso")
    except Exception:
        print("Error computing Cardoso simplicity:")
        traceback.print_exc()
        simplicity_cardoso = math.nan

    try:
        simplicity_cyclomatic = pm4py.simplicity_petri_net(net, im, fm, variant="extended_cyclomatic")
    except Exception:
        print("Error computing Cyclomatic simplicity:")
        traceback.print_exc()
        simplicity_cyclomatic = math.nan"""

    try:
        simplicity_entropy = entropy_simplicity_petri_net(net)
    except Exception:
        print("Error computing entropy-based simplicity:")
        traceback.print_exc()
        simplicity_entropy = {'entropy': math.nan, 'simplicity': math.nan}

    try:
        simplicity_size = size_simplicity_metric(net)
    except Exception:
        print("Error computing size-based simplicity:")
        traceback.print_exc()
        simplicity_size = math.nan

    print("\nSimplicity:")
    print(f"  simplicity_arc_degree: {simplicity_arc:.4f}" if isinstance(simplicity_arc, (float, int)) else f"  simplicity_arc_degree: {simplicity_arc}")
    #print(f"  simplicity_cardoso: {simplicity_cardoso:.4f}" if isinstance(simplicity_cardoso, (float, int)) else f"  simplicity_cardoso: {simplicity_cardoso}")
    #print(f"  simplicity_cyclomatic: {simplicity_cyclomatic:.4f}" if isinstance(simplicity_cyclomatic, (float, int)) else f"  simplicity_cyclomatic: {simplicity_cyclomatic}")
    print(f"  entropy: {simplicity_entropy['entropy']:.4f}")
    print(f"  simplicity_entropy: {simplicity_entropy['simplicity']:.4f}")
    print(f"  simplicity_size: {simplicity_size:.4f}")

    # ----------------------------
    # Generalization
    # ----------------------------
    try:
        generalization = pm4py.generalization_tbr(log, net, im, fm)
        print("\nGeneralization:")
        print(f"  generalization: {generalization:.4f}")
    except Exception:
        print("Error computing generalization:")
        traceback.print_exc()
        generalization = math.nan

    print("\n" + "-" * 50 + "\n")
